In [ ]:
# gemini model -> json parsing output -> function router -> response

In [13]:
from IPython.display import display, Markdown

In [1]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/arjun/Documents/github/research-agent/llmnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")

In [45]:
# Define your desired data structure.
class Weather(BaseModel):
    city: str = Field(description="city of weather user is interested in. na if not enough info")
    day: str = Field(description="the day of weather user is interested in. na if not enough info")
    additional_question: str = Field(description="additional question needed to answer inquiry. na if not needed")


In [46]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "what's the weather today?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Weather)

prompt = PromptTemplate(
    template="Answer the user query. If user doesn't give enough info, please request more info\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

out = chain.invoke({"query": joke_query})

In [47]:
out

{'city': 'na', 'day': 'today', 'additional_question': 'na'}

In [48]:
# pass arguments to a function router, i.e. tool is selected. -> passes through router -> appropriate func called